# Demonstration of the CNN used for time series classification

## Data preparation

In order to feed data to our model, we need to turn it into a keras sequence. This way it is fed to the network as batches of 1 sample each time, and there is no issue with the fact the time series have a different size. The whole preparation is done in the data_preparation function. The data are also shuffled.

In [1]:
from cnn import Turtlebot_CNN,Turtlebot_LSTM,data_preparation
from read_data import make_dataset,read_bag
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from inception import Inception


ds,l=data_preparation(path='/media/louis/TOSHIBA EXT/FullData')


[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/43/auto_sending_goals_2021-06-30-10-06-18 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/1/auto_sending_goals_2021-06-29-10-22-41 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/10/auto_sending_goals_2021-06-29-10-30-24 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/11/auto_sending_goals_2021-06-29-10-30-48 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/12/auto_sending_goals_2021-06-29-10-33-03 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/13/auto_sending_goals_2021-06-29-10-35-19 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/14/auto_sending_goals_2021-06-29-10-37-34 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/1

In [2]:
ds=ds.shuffle(l)
prop_train_val=0.8
l_train_val=math.ceil(prop_train_val*l)
train_val=ds.take(l_train_val)
test=ds.skip(l_train_val)
prop_train=0.8
l_train=math.floor(prop_train*l_train_val)
train=train_val.take(l_train)
validation=train_val.skip(l_train)
print(l_train)

115


In [3]:
train=train.batch(10)
test=test.batch(1)
validation=validation.batch(1)

In [4]:
inc=Inception()
cb_list_inc=[tf.keras.callbacks.ModelCheckpoint('inc_model/checkpoint_inc.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)]#tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=50),
inc.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[tf.keras.metrics.CategoricalAccuracy()])
hist_inc=inc.fit(train,epochs=1,callbacks=cb_list_inc,verbose=1,validation_data=validation)

(None, None, 13)


TypeError: in user code:

    /home/louis/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/louis/model_cnn/inception.py:86 call  *
        od=inc(od)
    /home/louis/model_cnn/inception.py:21 call  *
        x=self.bottlneck(input)
    /home/louis/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:1012 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    /home/louis/.local/lib/python3.8/site-packages/tensorflow/python/keras/layers/convolutional.py:248 call
        outputs = self._convolution_op(inputs, self.kernel)
    /home/louis/.local/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /home/louis/.local/lib/python3.8/site-packages/tensorflow/python/ops/nn_ops.py:1013 convolution_v2
        return convolution_internal(
    /home/louis/.local/lib/python3.8/site-packages/t

## Training the Model

We can now train the model using the fit method.

In [ ]:

lstm=Turtlebot_LSTM()
cb_list_lstm=[tf.keras.callbacks.ModelCheckpoint('lstm_model/checkpoint_lstm_new.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)]#tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=50),
lstm.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[tf.keras.metrics.CategoricalAccuracy()])
hist_lstm=lstm.fit(train,epochs=1,callbacks=cb_list_lstm,verbose=1,validation_data=validation)

12/12 [==============================] - 11s 853ms/step - loss: 1.1033 - categorical_accuracy: 0.2961 - val_loss: 1.0463 - val_categorical_accuracy: 0.4138
Model: "turtlebot_lstm_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                multiple                  5888      
_________________________________________________________________
lstm_5 (LSTM)                multiple                  50304     
_________________________________________________________________
concatenate_15 (Concatenate) multiple                  0         
_________________________________________________________________
dense_7 (Dense)              multiple                  4160      
_________________________________________________________________
dense_8 (Dense)              multiple                  8320      
_________________________________________________________________
dense_9 (Dense)           

In [ ]:
lstm.load_weights('lstm_model/checkpoint_lstm.h5')
plt.figure()
plt.plot(hist_lstm.history['loss'],label='Training Loss')
plt.plot(hist_lstm.history['val_loss'],label='Validation Loss')
plt.plot(hist_lstm.history['categorical_accuracy'],label='Training Accuracy')
plt.plot(hist_lstm.history['val_categorical_accuracy'],label='Validation Accuracy')
plt.legend()
plt.show()
lstm.evaluate(test)

In [5]:
cnn=Turtlebot_CNN()
cb_list_cnn=[tf.keras.callbacks.ModelCheckpoint('cnn_model/checkpoint_cnn_new.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)]#,tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=50)]
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[tf.keras.metrics.CategoricalAccuracy()])
hist_cnn=cnn.fit(train,epochs=500,callbacks=cb_list_cnn,verbose=1,validation_data=validation)
cnn.summary()

Epoch 1/500
(None, None, 13)


StagingError: in user code:

    /home/louis/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/louis/model_cnn/cnn.py:34 call  *
        x=self.concat([x1,x2])
    /home/louis/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:1008 __call__  **
        self._maybe_build(inputs)
    /home/louis/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:2710 _maybe_build
        self.build(input_shapes)  # pylint:disable=not-callable
    /home/louis/.local/lib/python3.8/site-packages/tensorflow/python/keras/utils/tf_utils.py:272 wrapper
        output_shape = fn(instance, input_shape)
    /home/louis/.local/lib/python3.8/site-packages/tensorflow/python/keras/layers/merge.py:500 build
        del reduced_inputs_shapes[i][self.axis]

    IndexError: list assignment index out of range


In [ ]:
cnn.load_weights('cnn_model/checkpoint_cnn_new.h5')

plt.figure()
plt.plot(hist_cnn.history['loss'],label='Training Loss')
plt.plot(hist_cnn.history['val_loss'],label='Validation Loss')
plt.plot(hist_cnn.history['categorical_accuracy'],label='Training Accuracy')
plt.plot(hist_cnn.history['val_categorical_accuracy'],label='Validation Accuracy')
plt.legend()
plt.show()
cnn.evaluate(test)

## Testing the model

Here is a Visualization of what the CNN does 

![Visualization of the CNN](img/Schema_CNN.png)

In [ ]:
import numpy as np
def results(model,test):
    res=model.predict(test)
    c=0
    for i,pred in enumerate(res):
        y=test[i][1][0]
        if np.argmax(y)==np.argmax(pred):
            c+=1
    return(c/len(res))
            

In [ ]:
lstm.load_weights('lstm_model/checkpoint_lstm.h5')
cnn.load_weights('cnn_model/checkpoint_cnn.h5')
print('LSTM :',results(lstm,test))
print('CNN :',results(cnn,test))

In [ ]:
model.predict(test)

In [ ]:
train[0]

In [ ]:
tf.data.Dataset.from_generator(train)

In [ ]:
odoms,scans,labels=make_dataset('/media/louis/TOSHIBA EXT/FullData')

In [ ]:
tf.data.Dataset.from_generator(lambda :odoms,output_shapes=[None,13],output_types=tf.float32)

In [ ]:
np.shape(odoms[1])

In [ ]:
lstm=Turtlebot_CNN(type='lstm')
lstm.compile(optimizer='adam',loss='categorical_crossentropy')
lstm.load_weights('lstm_model/checkpoint_lstm.h5')
lstm.predict(train)

In [ ]:
for element in train:
    print(element)

In [ ]:
import tensorflow as tf
import numpy as np
from read_data import make_dataset

odoms,scans,labels=make_dataset("/media/louis/TOSHIBA EXT/FullData")


[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/43/auto_sending_goals_2021-06-30-10-06-18 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/1/auto_sending_goals_2021-06-29-10-22-41 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/10/auto_sending_goals_2021-06-29-10-30-24 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/11/auto_sending_goals_2021-06-29-10-30-48 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/12/auto_sending_goals_2021-06-29-10-33-03 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/13/auto_sending_goals_2021-06-29-10-35-19 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/14/auto_sending_goals_2021-06-29-10-37-34 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/1

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
tf.data.Dataset.from_tensor_slices(odoms)

list

In [ ]:
list_data=[np.random.random((np.random.randint(200,500),13)).tolist() for i in range(10)]
test=tf.ragged.constant(list_data,ragged_rank=1)
ds=tf.data.Dataset.from_tensor_slices(test)

plop
(303, 13)
plop
(417, 13)
plop
(481, 13)
plop
(448, 13)
plop
(364, 13)
plop
(331, 13)
plop
(222, 13)
plop
(454, 13)
plop
(448, 13)
plop
(378, 13)
